In [1]:
from ml_recon.Models.modl import modl
from torch.utils.data import DataLoader
from ml_recon.Transforms import (pad, trim_coils, combine_coil, toTensor, permute, 
                        view_as_real, remove_slice_dim, fft_2d, normalize, addChannels)
from ml_recon.Dataset.undersampled_dataset import UndersampledKSpaceDataset
from torchvision.transforms import Compose
import numpy as np

import torch
from ml_recon.Utils import image_slices, save_model
from ml_recon.Utils.collate_function import collate_fn
from ml_recon.Models.varnet import VarNet

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
transforms = Compose(
    (
        pad((640, 320)), 
        toTensor(),
        normalize(),
    )
)
dataset = UndersampledKSpaceDataset('/home/kadotab/projects/def-mchiew/kadotab/Datasets/fastMRI/multicoil_train', transforms=transforms, R=4)
dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)
    

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
model = VarNet(2, 2, num_cascades=12, use_norm=True)
model.to(device)

VarNet(
  (cascade): ModuleList(
    (0): VarnetBlock(
      (unet): Unet(
        (down_sample_layers): ModuleList(
          (0): double_conv(
            (conv1): Conv2d(2, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv2): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (activation): LeakyReLU(negative_slope=0.2, inplace=True)
            (instance_norm1): InstanceNorm2d(18, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (instance_norm2): InstanceNorm2d(18, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (drop_out1): Dropout2d(p=0, inplace=False)
            (drop_out2): Dropout2d(p=0, inplace=False)
          )
          (1): Unet_down(
            (down): down(
              (max_pool): AvgPool2d(kernel_size=2, stride=(2, 2), padding=0)
            )
            (conv): double_conv(
              (conv1): Conv2d(18, 36, kernel_size=(3, 3), st

In [8]:
checkpoint = torch.load('/home/kadotab/python/ml/ml_recon/Model_Weights/20230414-082106VarNet.pt', map_location=torch.device('cpu'))

In [12]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [13]:
model.eval()

VarNet(
  (cascade): ModuleList(
    (0): VarnetBlock(
      (unet): Unet(
        (down_sample_layers): ModuleList(
          (0): double_conv(
            (conv1): Conv2d(2, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv2): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (activation): LeakyReLU(negative_slope=0.2, inplace=True)
            (instance_norm1): InstanceNorm2d(18, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (instance_norm2): InstanceNorm2d(18, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (drop_out1): Dropout2d(p=0, inplace=False)
            (drop_out2): Dropout2d(p=0, inplace=False)
          )
          (1): Unet_down(
            (down): down(
              (max_pool): AvgPool2d(kernel_size=2, stride=(2, 2), padding=0)
            )
            (conv): double_conv(
              (conv1): Conv2d(18, 36, kernel_size=(3, 3), st

In [14]:
data = next(iter(dataloader))

In [20]:
output = model(data['undersampled'][[5], :, :, :], data['mask'][[5], :, :])

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 262144000 bytes. Error code 12 (Cannot allocate memory)